In [1]:
import pyspark

In [2]:
import sys 
sys.path.insert(0, '/home/rnshishir/deepmed/CPRD_Cut22/')
import shutil
from utils.yaml_act import yaml_load
from utils.arg_parse import arg_paser
from CPRD.config.spark import spark_init, read_parquet
import pyspark.sql.functions as F
from CPRD.functions import tables, merge
from CPRD.functions import merge
from CPRD.functions.merge import *

from utils.utils import save_obj
from CPRD.functions.MedicalDictionary import * 
from CPRD.functions.Prediction import * 
from CPRD.functions.cohort_select import * 
from CPRD.functions.SurvPrediction import * 

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [3]:

def vocabCreate(CountsICD, num):
    dic={}
    dic['token2idx']={}
    dic['idx2token']={}
    dic['token2idx']['MASK']=4
    dic['token2idx']['CLS']=3
    dic['token2idx']['SEP']=2
    dic['token2idx']['UNK']=1
    dic['token2idx']['PAD']=0
    dic['idx2token'][4]='MASK'
    dic['idx2token'][3]='CLS'
    dic['idx2token'][2]='SEP'
    dic['idx2token'][1]='UNK'
    dic['idx2token'][0]='PAD'
    i=5
    for x in CountsICD:
        if CountsICD[x]>num:
            dic['token2idx'][x]=i
            dic['idx2token'][i]=x
            i=i+1
    return dic

In [4]:
args = dotdict({'params': '/home/rnshishir/deepmed/CPRD_Cut22/config/config.yaml'})
params = yaml_load(args.params)
spark_params = params['pyspark']
spark = spark_init(spark_params)
file = params['file_path']
data_params = params['params']

/home/rnshishir/deepmed/CPRD_Cut22/utils/yaml_act.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [5]:
# params have each of the individual files for data processing and other things like location of medical dict etc
params

{'task': 'medication.main',
 'pyspark': {'temp': '/home/rnshishir/sparkNewT',
  'pyspark_env': '/home/rnshishir/anaconda3/envs/pyspark37/bin/python3.7'},
 'file_path': {'clinical': '/home/workspace/datasets/cprd/cprd2021/*/*_Observation_*',
  'patient': '/home/workspace/datasets/cprd/cprd2021/*/*_Patient_*',
  'problem': '/home/workspace/datasets/cprd/cprd2021/*/*_Problem_*',
  'therapy': '/home/workspace/datasets/cprd/cprd2021/*/*_DrugIssue_*',
  'referral': '/home/workspace/datasets/cprd/cprd2021/*/*_Referral_*',
  'practice': '/home/workspace/datasets/cprd/cprd2021/*/*_Practice_*',
  'consultation': '/home/workspace/datasets/cprd/cprd2021/*/*_Consultation_*',
  'staff': '/home/workspace/datasets/cprd/cprd2021/*/*_Staff*',
  'diagnosis_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_diagnosis_hosp_20_095_DM.txt',
  'proc_hes': '/home/workspace/datasets/cprd/cprd2021/linkage/20_095_Results/*/Aurum_linked/Final/hes_procedures_epi_20_095_DM.

# Predictor extraction...
### maybe predictors for the (part of the) qrisk3 model!

In [6]:
allDiag = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
meds = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/medications_1985_2022_bnfvtm.parquet')
proc = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/proc_HES_1985_2021_allpats.parquet').withColumnRenamed('OPCS','code')

proc = proc.withColumn('code', F.concat(F.lit("PROC_"), F.col('code'))).select('patid','eventdate','code')

allDiag = allDiag.select('patid','eventdate','ICD').withColumnRenamed('ICD','code')
meds = meds.select('patid','eventdate','code')
fullcodes = allDiag.union(meds).union(proc)

In [7]:
# sample cohort with 100 patients or so
cohort = read_parquet(spark.sqlContext,'predextract_cohort_demo.parquet')

In [8]:
cohort.count()

101

In [9]:
from CPRD.functions.predictor_extractor import *

In [10]:
predictor = PredictorExtractorBase()
md = MedicalDictionaryRiskPrediction(file, spark)

# Cont and categorical variable extraction
### variable name is show in each cell
#### Lookback before baseline is span_before_baseline_month, and what you want is type: last, mean, std, etc... examples are self explanatory

In [ ]:
# bmi extract
bmi = read_parquet(spark.sqlContext ,'/home/shared/shishir/AurumOut/rawDat/bmi_1985_2021.parquet')
bmi = predictor.predictor_extract(bmi, cohort, 'BMI',colname = 'bmi', col_baseline='study_entry', span_before_baseline_month=36, type='last')
cohort = cohort.join(bmi, 'patid', 'left')



cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv1'+"test.parquet")


In [ ]:
# smoking status extract
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv1'+"test.parquet")
print(cohort.count())
smoke = read_parquet(spark.sqlContext ,'/home/shared/shishir/AurumOut/rawDat/smoklevel_1985_2021.parquet')
smoke = predictor.predictor_extract(smoke, cohort, 'smoke',colname = 'smoke' , col_baseline='study_entry', span_before_baseline_month=1000,type='last')
cohort = cohort.join(smoke, 'patid', 'left')

cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv2'+"test.parquet")




In [ ]:
# tchdl extract
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv2'+"test.parquet")

tchdl = read_parquet(spark.sqlContext ,'/home/shared/shishir/AurumOut/rawDat/tchdl_rat_1985_2021.parquet')
# tchdl = retrieve_smoking_status(file, spark, duration=(1985, 2021))
tchdl = predictor.predictor_extract(tchdl, cohort, 'tchdl_rat',colname = 'tchdl' , col_baseline='study_entry', span_before_baseline_month=36,type='mean')
cohort = cohort.join(tchdl, 'patid', 'left')

cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv3'+"test.parquet")




In [ ]:
# bp extract
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv3'+"test.parquet")
print(cohort.count())

bp = read_parquet(spark.sqlContext ,'/home/shared/shishir/AurumOut/rawDat/AllBP_praclinkage_1985_2021.parquet')
bp = predictor.predictor_extract(bp, cohort, 'systolic',colname= 'systolic', col_baseline='study_entry', span_before_baseline_month=1000,type='last')
cohort = cohort.join(bp, 'patid', 'left')

cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv4'+"test.parquet")


In [ ]:
# interesting one :) - this is systolic blood pressure Standard Deviation extraction, yes, you got that right, i have also coded this!
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv4'+"test.parquet")
print(cohort.count())

bp = read_parquet(spark.sqlContext ,'/home/shared/shishir/AurumOut/rawDat/AllBP_praclinkage_1985_2021.parquet')
bp = predictor.predictor_extract(bp, cohort, 'systolic',colname= 'sysvar', col_baseline='study_entry', span_before_baseline_month=36,type='std')
cohort = cohort.join(bp, 'patid', 'left')

cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv5'+"test.parquet")


In [ ]:
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv5'+"test.parquet")
cohort = pd.read_parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv5'+"test.parquet")


# Disease/medication/procedure at baseline variable extraction
### variable name is show in each cell


In [13]:
# generic... rheumaatoid arthritis
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv5'+"test.parquet")
print(cohort.count())
rheum = md.queryDisease(md.findItem('rheumatoid art'), merge= True)['rheumatoid arthritis']
allcoderheum = list(rheum['ICD10']) + list(rheum['medcode'])
print(allcoderheum)

allDiag_preserve = split_combine_diag (spark , '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')

rheum = predictor.predictor_check_exist(allcoderheum, allDiag_preserve, cohort, col='code', col_baseline='study_entry').withColumnRenamed('code', 'rheum')
cohort = cohort.join(rheum, 'patid', 'left')
cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv6'+"test.parquet")


101
['M0630', 'M0534', 'M0500', 'M0507', 'M0590', 'M0506', 'M0508', 'M0524', 'M0580', 'M0585', 'M0638', 'M0699', 'M0504', 'M0698', 'M0598', 'M0502', 'M0593', 'M0696', 'M0625', 'M0589', 'M0603', 'M0623', 'M080', 'M0588', 'M0522', 'M0595', 'M0689', 'M0803', 'M0607', 'M0510', 'M0530', 'M0631', 'M0686', 'M062', 'M0519', 'M05', 'M0601', 'M0501', 'M0634', 'M0639', 'M0521', 'M0694', 'M063', 'M058', 'M0606', 'M0809', 'M0685', 'M0636', 'M0528', 'M0691', 'M0532', 'M0695', 'M0806', 'M0802', 'M0801', 'M0539', 'M0514', 'M0628', 'M0692', 'M0687', 'M0527', 'M0596', 'M0682', 'M0513', 'M0520', 'M0525', 'M0526', 'M0633', 'M0533', 'M0684', 'M0581', 'M0690', 'M050', 'M0804', 'M059', 'M0516', 'M0517', 'M0591', 'M0594', 'M0509', 'M068', 'M0608', 'M053', 'M0627', 'M0584', 'M0523', 'M0537', 'M0583', 'M0609', 'M0624', 'M0503', 'M0535', 'M0688', 'M0518', 'M0531', 'M0592', 'M060', 'M0635', 'M0600', 'M0529', 'M0602', 'M0605', 'M0538', 'M0808', 'M0632', 'M0800', 'M0586', 'M0604', 'M0512', 'M052', 'M0511', 'M0597',

In [23]:
# this is a tutorial for a complex disease extraction; traeated hypertension = not just hypertension, but ALSO treatment for hypertension

cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv6'+"test.parquet")


# get codees for hypertension
hyptn = md.queryDisease(md.findItem('hypertension'), merge= True)['merged']
allcodeHyperten = list(hyptn['ICD10']) + list(hyptn['medcode']) 
print(allcodeHyperten)

# get codes for antihyp
prodcodesAntiHyp  = md.queryMedication(md.findItem('hyperte'), merge= True)['merged']['prodcode']
print(prodcodesAntiHyp)

# load diag files and med files
allDiag_preserve = split_combine_diag (spark , '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
meds = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/medications_1985_2022_bnfvtm.parquet')[['patid','eventdate','prodcode']].withColumnRenamed('prodcode','code')


# extract diag hyp
Hyperten = predictor.predictor_check_exist(allcodeHyperten, allDiag_preserve, cohort, col='code', col_baseline='study_entry').withColumnRenamed('code', 'hyp')
cohort = cohort.join(Hyperten, 'patid', 'left')

# extract med antihyp
antiHyperten = predictor.predictor_check_exist(prodcodesAntiHyp, meds, cohort, col='code', col_baseline='study_entry').withColumnRenamed('code', 'antihyp')
cohort = cohort.join(antiHyperten, 'patid', 'left')

# logic statement
cohort = cohort.withColumn('treat_hyp',  F.when(((F.col("hyp") ==1) &(F.col('antihyp')==1)) , 1)     .otherwise(0))
cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv7'+"test.parquet")




['K766', 'I272', 'I270', 'G932', 'I119', 'I10', 'I12', 'I13', 'I158', 'I11', 'I152', 'P292', 'I15', 'I110', 'G932', 'I674', 'I159', 'I150', '5057801000006113', '3552511000006116', '741661000006118', '99047018', '76661000006110', '3117511000006117', '64282015', '443764015', '909441000006118', '3468501000006117', '523801000006119', '26091000000116', '4356081000006115', '2566081000006118', '264486018', '351361000000117', '76651000006113', '728671000006119', '76631000006118', '1780252014', '4775791000006113', '3117461000006115', '300870016', '299687010', '158241000006117', '3552501000006119', '3950631000006119', '99042012', '395751018', '3784371000006115', '3117411000006118', '151161000006115', '2193021000000110', '3296351000006114', '2193971000000110', '741701000006114', '1806071000006118', '350601000000116', '299680012', '19411000006110', '316848016', '3552521000006112', '76671000006115', '4775861000006110', '3768511000006114', '143003017', '90135019', '884121000006111', '184696100000611

In [ ]:
# antipsychotic extraction with CUSTOM codes!
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv7'+"test.parquet")
# extract antispysch codes
antipsychcodesdrug = pd.read_csv('antispych.txt').prodcodeid.values
antipsychcodesdrug = [str(x) for x in antipsychcodesdrug]

allcodeantipsych = antipsychcodesdrug
print(allcodeantipsych)

# load medications///
meds = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/medications_1985_2022_bnfvtm.parquet')[['patid','eventdate','prodcode']].withColumnRenamed('prodcode','code')

antipsych = predictor.predictor_check_exist(allcodeantipsych, meds, cohort, col='code', col_baseline='study_entry').withColumnRenamed('code', 'antipsych')
cohort = cohort.join(antipsych, 'patid', 'left')
cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv8'+"test.parquet")


In [ ]:
# fam history extraction
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv8'+"CVD_survout_nostatin_withImd_withpracid_wValidstatus_1000GPsubset_cohort_25_84_2000_2020.parquet")
famhis = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/famhistoryCHD_all.parquet')

famhistorycodes = pd.read_csv('Familyhistorycoronaryheartdisease.csv')
meddict = pd.read_table('/home/workspace/datasets/cprd/cprd2021/202102_lookups/202102_EMISMedicalDictionary.txt')
famhistorycodes=famhistorycodes.Code.values
famhistorygpcode = meddict[meddict.SnomedCTConceptId.isin(famhistorycodes)].MedCodeId.values
famhistorygpcode = [str(x) for x in famhistorygpcode]
allcodefamhistory = list(famhistorygpcode)



# get fam history data
famhis = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/famhistoryCHD_all.parquet').withColumnRenamed('medcode','code')
famhis= famhis.select('patid','eventdate','code')
print(allcodefamhistory)

# extarct chd fam history particularly...
famhistory = predictor.predictor_check_exist(allcodefamhistory, famhis, cohort, col='code', col_baseline='enddate').withColumnRenamed('code', 'famhistory_chd')
cohort = cohort.join(famhistory, 'patid', 'left')
cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv9'+"test.parquet")


In [ ]:
# IMD and Ethnicity extraction!!

cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv9'+"CVD_survout_nostatin_withImd_withpracid_wValidstatus_1000GPsubset_cohort_25_84_2000_2020.parquet")

imd = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/imd_allPats.parquet')
eth = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/rawDat/raw_eth_linkedpat.parquet')

cohort = cohort.join(eth,'patid','left')
cohort = cohort.join(imd , 'patid','left')
cohort.write.parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv10'+"CVD_survout_nostatin_withImd_withpracid_wValidstatus_1000GPsubset_cohort_25_84_2000_2020.parquet")


In [ ]:
cohort = pd.read_parquet('/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv10'+"CVD_survout_nostatin_withImd_withpracid_wValidstatus_1000GPsubset_cohort_25_84_2000_2020.parquet")


In [ ]:
for x in [ 'rheum', 'hyp', 'antihyp', 'treat_hyp',  'antipsych', 'famhistory_chd',
       ]:
    print(x, cohort[x].mean())

# T-BEHRT/BEHRT/TRisk etc data extraction 
### also option to make unique per N months (i.e., delete duplicate rec in a given N months)

In [37]:
allDiag = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
meds = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/medications_1985_2022_bnfvtm.parquet')
proc = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/proc_HES_1985_2021_allpats.parquet').withColumnRenamed('OPCS','code')

proc = proc.withColumn('code', F.concat(F.lit("PROC_"), F.col('code'))).select('patid','eventdate','code')

allDiag = allDiag.select('patid','eventdate','ICD').withColumnRenamed('ICD','code')
meds = meds.select('patid','eventdate','code')
fullcodes = allDiag.union(meds).union(proc)


In [34]:
cohort.columns

['patid',
 'gender',
 'dob',
 'study_entry',
 'startdate',
 'enddate',
 'exp_label',
 'eventdate',
 'label',
 'time2event']

In [35]:
cohort = read_parquet(spark.sqlContext, 'test9_40m.parquet')

cohort = cohort[['patid',
 'gender',
 'dob',
 'study_entry',
 'startdate',
 'enddate',
 'exp_label',
 'label',
 'time2event']]

extractorBEHRT = BEHRTextraction()
dataout = extractorBEHRT.format_behrt(fullcodes, cohort, col_entry = 'study_entry', unique_in_months=6)

In [36]:
dataout.write.parquet('test9_40m__BEHRTbasedout.parquet')


In [38]:
dataout = pd.read_parquet('test9_40m__BEHRTbasedout.parquet')

# custom extraction extra set of codes examples...

In [ ]:
# FOR CUSTOM EXTRACTION.... rheumaatoid arthritis
cohort = read_parquet(spark.sqlContext,'/home/shared/shishir/AurumOut/CVDPred/'+'statsmodel_prepv5'+"CVD_survout_nostatin_withImd_withpracid_wValidstatus_1000GPsubset_cohort_25_84_2000_2020.parquet")

rheumcodes = pd.read_csv('RheumatoidArthritis.csv')
meddict = pd.read_table('/home/workspace/datasets/cprd/cprd2021/202102_lookups/202102_EMISMedicalDictionary.txt')
rheumcodes=rheumcodes.Code.values
rheumgpcode = meddict[meddict.SnomedCTConceptId.isin(rheumcodes)].MedCodeId.values
rheumgpcode = [str(x) for x in rheumgpcode]
rheumcodeicd10 = pd.read_csv('Rheumatoidarthritis(ICD10).csv')
rheumcodeicd10 = rheumcodeicd10.Code.values

allcoderheum = list(rheumcodeicd10) + list(rheumgpcode)
print(allcoderheum)
allDiag = read_parquet(spark.sqlContext, '/home/shared/shishir/AurumOut/rawDat/diagGP_med2sno2icd_HESAPC_praclinkage_1985_2021.parquet')
GPdiags = allDiag[allDiag.source=='CPRD']
GPdiags = GPdiags.select(['patid' , 'eventdate', 'medcode','source']).withColumnRenamed('medcode', 'code')
HESdiags = allDiag[allDiag.source=='HES']
HESdiags = HESdiags.select(['patid' , 'eventdate', 'ICD', 'source']).withColumnRenamed('ICD', 'code')
allDiag_preserve= GPdiags.union(HESdiags).select('patid','eventdate','code')

rheum = predictor.predictor_check_exist(allcoderheum, allDiag_preserve, cohort, col='code', col_baseline='study_entry').withColumnRenamed('code', 'rheum')
cohort = cohort.join(rheum, 'patid', 'left')